In [3]:
from __future__ import absolute_import, print_function

import numpy as np
import pyopencl as cl
import os

ImportError: DLL load failed: The specified procedure could not be found.

In [2]:
os.environ['PYOPENCL_CTX'] = ':3'

NameError: name 'os' is not defined

In [ ]:
a_np = np.random.rand(50000).astype(np.float32)
b_np = np.random.rand(50000).astype(np.float32)

ctx   = cl.create_some_context()
queue = cl.CommandQueue(ctx)

In [ ]:
mf   = cl.mem_flags
a_g  = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a_np)
b_g  = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b_np)

In [ ]:
prg = cl.Program(ctx, """
__kernel void sum(
   __global const float *a_g, __global const float *b_g, __global float *res_g)
{
  int gid    = get_global_id(0);
  res_g[gid] = a_g[gid] + b_g[gid];
}
""").build()

In [ ]:
res_g = cl.Buffer(ctx, mf.WRITE_ONLY, a_np.nbytes)
prg.sum(queue, a_np.shape, None, a_g, b_g, res_g)

In [ ]:
res_np = np.empty_like(a_np)
cl.enqueue_copy(queue, res_np, res_g)

In [ ]:
# Check on GPU with Numpy:
print(res_np)

In [ ]:
print(a_np + b_np)

In [ ]:
assert np.allclose(res_np, a_np + b_np)